In [2]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

/home/du3/Desktop/duchatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = pd.read_parquet('train-00000-of-00001-1ae224438dce829b.parquet')
dataset
results_df = pd.DataFrame(columns=["instruction", "reference", "generated", "meteor_score"])

In [5]:
def generate_response(model, tokenizer, prompt, max_length=256):
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Initialize the progress bar
    total_steps = max_length  # Number of decoding steps corresponds to max_length
    with tqdm(total=total_steps, desc="Generating response", unit="step") as pbar:
        # Generate the response
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=5,
            early_stopping=True,
            # Update the progress bar on each decoding step
            output_scores=True, 
            return_dict_in_generate=True,
        )
        
        # Update the progress bar
        for _ in range(len(outputs.sequences[0])):
            pbar.update(1)
    
    # Decode the generated response
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    return response

In [6]:
from nltk.translate.meteor_score import meteor_score

def evaluate_model(dataset, model, tokenizer):
    global results_df
    meteor_scores = []
    for idx in range(len(dataset)):
        instruction = dataset.at[idx, "instruction"]
        reference = dataset.at[idx, "output"] 
        generated_text = generate_response(model, tokenizer, instruction)

        # METEOR 점수 계산
        meteor_score_value = meteor_score([reference], generated_text)
        meteor_scores.append(meteor_score_value)

        print(f"Input: {instruction}")
        print(f"Reference: {reference}")
        print(f"Generated: {generated_text}")
        print(f"METEOR Score: {meteor_score_value}\n")
        
        # DataFrame에 결과 추가
        results_df = pd.concat([results_df, pd.DataFrame({
            "instruction": [instruction],
            "reference": [reference],
            "generated": [generated_text],
            "meteor_score": [meteor_score_value]
        })], ignore_index=True)

    # CSV 파일로 저장
    results_df.to_csv("EEVE-Korean-Instruct-10.8B-v1.0_evaluation_meteor.csv", index=False)


In [ ]:
if __name__ == "__main__":
    parquet_path = "train-00000-of-00001-1ae224438dce829b.parquet"  
    model_path = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"  

    model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
    tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")

    # 평가 실행
    evaluate_model(dataset, model, tokenizer)